In [27]:
%pip install boto3 termcolor -U

from termcolor import colored

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 7.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.40
    Uninstalling botocore-1.35.40:
      Successfully uninstalled botocore-1.35.40
  Attempting uninstall: boto3
    Found existing installation: boto3 1.35.40
    Uninstalling boto3-1.35.40:
      Successfully uninstalled boto3-1.35.40
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scoutsuite 5.14.0 requires python-dateutil<2.8.1,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Define tools

In [28]:

class StationNotFoundError(Exception):
    """Raised when a radio station isn't found."""
    pass

def get_top_song(sign):
    """Returns the most popular song for the requested station.
    Args:
        call_sign (str): The call sign for the station for which you want
        the most popular song.

    Returns:
        response (json): The most popular song and artist.
    """

    song = ""
    artist = ""
    if sign == 'WZPZ':
        song = "Elemental Hotel"
        artist = "8 Storey Hike"
    elif sign == 'WZHK':
        song = "Elemental Hotel"
        artist = "8 Storey Hike"
    else:
        raise StationNotFoundError(f"Station {sign} not found.")

    return {"json": {"song": song, "artist": artist}}

tools_imp = {
    'top_song': get_top_song,
}


tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "top_song",
                "description": "Get the most popular song played on a radio station.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "sign": {
                                "type": "string",
                                "description": "The call sign for the radio station for which you want the most popular song. Example calls signs are WZPZ, and WKRP."
                            }
                        },
                        "required": [
                            "sign"
                        ]
                    }
                }
            }
        }
    ]
}

def invoke_tool(tool):
    print('\nInvoking tool: %s. params: %s. id: %s' % (tool['name'], tool['input'], tool['toolUseId']))
    if tool['name'] in tools_imp:
        tool_result = tools_imp[tool['name']](**tool['input'])
        print("Tool invoke id: %s, result: %s" % (tool['toolUseId'], tool_result))
        return tool_result
    else:
        print(f"Tool {tool['name']} not found.")
        return None


## 流式模式使用 Tool use 功能

In [29]:
"""
Shows how to use the Conversation API to stream a response from Anthropic Claude 3 (on demand).
"""

import logging
import boto3
import json

from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def print_metadata(metadata):
    """
    print metadata of streaming response: such as in, out and total tokens, latency(if has)

    Args:
        None
    
    Returns:
        None
    """
    if 'usage' in metadata:
        print("\nToken usage")
        print(f"Input tokens: {metadata['usage']['inputTokens']}")
        print(
            f":Output tokens: {metadata['usage']['outputTokens']}")
        print(f":Total tokens: {metadata['usage']['totalTokens']}")
    if 'metrics' in metadata:
        print(
            f"Latency: {metadata['metrics']['latencyMs']} milliseconds")


def stream_conversation(bedrock_client,
                    model_id,
                    messages,
                    system_prompts,
                    inference_config,
                    additional_model_fields,
                    tool_config,
                    round=0):
    """
    Sends messages to a model and streams the response.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        messages (JSON) : The messages to send.
        system_prompts (JSON) : The system prompts to send.
        inference_config (JSON) : The inference configuration to use.
        additional_model_fields (JSON) : Additional model fields to use.

    Returns:
        Nothing.

    """

    print(f'\n************ ROUND {round} START ************')
    print("Streaming messages with model %s" % model_id)

    bedrock_params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": inference_config,
        "additionalModelRequestFields": additional_model_fields,
        "toolConfig": tool_config
    }

    system = [item for item in system_prompts if item.get('text')]
    if system:
        bedrock_params['system'] = system

    response = bedrock_client.converse_stream( **bedrock_params )
    stream = response.get('stream')
    tools_buf = {}
    resp_text_buf = ''
    if stream:
        stop_reason = None
        metadata = None
        for event in stream:
            # print(event)
            if 'messageStart' in event:
                print(f"\nRole: {event['messageStart']['role']}")

            if 'contentBlockStart' in event:
                if 'toolUse' in event['contentBlockStart']['start']:
                    block_index = event['contentBlockStart']['contentBlockIndex']
                    tools_buf[block_index] = event['contentBlockStart']['start']['toolUse']
                    tools_buf[block_index]['input'] = ''
            if 'contentBlockDelta' in event:
                block_index = event['contentBlockDelta']['contentBlockIndex']
                delta_types = event['contentBlockDelta']['delta'].keys()
                if 'text' in delta_types:
                    text_delta = event['contentBlockDelta']['delta']['text']
                    print(colored(text_delta, 'green'), end="")
                    resp_text_buf += text_delta
                elif 'toolUse' in delta_types:
                    tools_buf[block_index]['input'] += event['contentBlockDelta']['delta']['toolUse']['input']
                    # print(event['contentBlockDelta']['delta']['toolUse']['input'], end="")

            if 'metadata' in event:
                metadata = event['metadata']

            if 'messageStop' in event:
                stop_reason = event['messageStop']['stopReason']
                # print(f"\nStop reason: { stop_reason }")

        # put assistant response into messages
        assistant_message = {
            "role": "assistant",
            "content": []
        }
        if resp_text_buf:
            assistant_message['content'].append({ "text": resp_text_buf })

        messages.append(assistant_message)

        # parse tool use response, invoke tools and trigger next inference automatically
        if 'tool_use' == stop_reason:
            # store tool use result
            tool_result_message = {
                "role": "user",
                "content": []
            }

            for tool in tools_buf.values():
                tool['input'] = json.loads(tool['input'])
                assistant_message['content'].append( {'toolUse': tool } )

                tool_result = {}
                try:
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [ invoke_tool(tool) ]
                    }
                except StationNotFoundError as err:
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"text":  err.args[0]}],
                        "status": 'error'
                    }
                tool_result_message['content'].append({"toolResult": tool_result})
            
            # put tool invocation result into messages as user message
            messages.append(tool_result_message)
            print_metadata(metadata)
            # trigger next inference
            stream_conversation(bedrock_client, model_id, messages, system_prompts, 
                                inference_config, additional_model_fields, tool_config, round + 1)
        else:
            print_metadata(metadata)
        print(f"----tools_buf----:\n{tools_buf}")
        return messages


In [41]:

def main():
    """
    Entrypoint for streaming message API response example.
    """

    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")

    # model_id = "anthropic.claude-3-haiku-20240307-v1:0"
    # model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    # model_id = "meta.llama3-1-70b-instruct-v1:0" 
    # model_id = "meta.llama3-1-8b-instruct-v1:0"
    # model_id = "cohere.command-r-v1:0" # doesn't support tool use in streaming mode
    model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"
keke    system_prompt = ""

    # Message to send to the model.
    input_text = "What is the most popular song on WZPZ and WZHK?"
    print(colored(f"Question: {input_text}", 'red'))

    message = {
        "role": "user",
        "content": [{"text": input_text}]
    }
    messages = [message]
    
    # System prompts.
    system_prompts = [{"text" : system_prompt}]

    # inference parameters to use.
    temperature = 0.9
    top_k = 200
    max_tokens = 8000
    # Base inference parameters.
    inference_config = {
        "temperature": temperature,
        "maxTokens": max_tokens,
    }
    # Additional model inference parameters.
    additional_model_fields = {"top_k": top_k}

    try:
        session = boto3.session.Session(profile_name='default',region_name='us-west-2')
        bedrock_client = session.client(service_name='bedrock-runtime')

        stream_conversation(bedrock_client, model_id, messages,
                        system_prompts, inference_config, additional_model_fields, tool_config)

    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        print("A client error occured: " +
              format(message))

    else:
        print(
            f"\nFinished streaming messages with model {model_id}.")

main()

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Question: What is the most popular song on WZPZ and WZHK?

************ ROUND 0 START ************
Streaming messages with model anthropic.claude-3-5-sonnet-20241022-v2:0


ERROR:__main__:A client error occurred: The maximum tokens you requested exceeds the model limit of 4096. Try again with a maximum tokens value that is lower than 4096.


A client error occured: The maximum tokens you requested exceeds the model limit of 4096. Try again with a maximum tokens value that is lower than 4096.


## 非流式模式使用 Tool use 功能

In [5]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to use tools with the Conversation API and the Claude model.
"""

import logging
import boto3
import json

from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def generate_text(bedrock_client, 
                  model_id, 
                  messages, 
                  system_prompts, 
                  inference_config, 
                  additional_model_fields, 
                  tool_config, 
                  round=0):
    """Generates text using the supplied Amazon Bedrock model. If necessary,
    the function handles tool use requests and sends the result to the model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The Amazon Bedrock model ID.
        tool_config (dict): The tool configuration.
        messages (list): The input messages.
    Returns:
        Nothing.
    """

    logger.info("Generating text with model %s", model_id)


    bedrock_params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": inference_config,
        "additionalModelRequestFields": additional_model_fields,
        "toolConfig": tool_config
    }

    system = [item for item in system_prompts if item.get('text')]
    if system:
        bedrock_params['system'] = system


    response = bedrock_client.converse(**bedrock_params)

    output_message = response['output']['message']
    messages.append(output_message)
    output_content = output_message['content']

    tool_list = []
    tool_result_message = {
        "role": "user",
        "content": []
    }

    for item in output_content:
        if 'text' in item:
            print(colored(item['text'], 'green'))

        if 'toolUse' in item:
            tool_list.append(item['toolUse'])

    for tool in tool_list:
        logger.info("Requesting tool %s. Request: %s",
                        tool['name'], tool['toolUseId'])

        tool_result = {}
        try:
            tool_result = {
                "toolUseId": tool['toolUseId'],
                "content": [ invoke_tool(tool) ]
            }
        except StationNotFoundError as err:
            tool_result = {
                "toolUseId": tool['toolUseId'],
                "content": [{"text":  err.args[0]}],
                "status": 'error'
            }

        tool_result_message['content'].append({
            "toolResult": tool_result
        })
    if tool_list:
        messages.append(tool_result_message)
        generate_text(bedrock_client, model_id, messages, system_prompts, 
                      inference_config, additional_model_fields, tool_config, round + 1)

In [37]:
def main():
    """
    Entrypoint for tool use example.
    """

    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    # inference parameters to use.
    temperature = 0.9
    top_k = 200
    max_tokens = 2000
    # Base inference parameters.
    inference_config = {
        "temperature": temperature,
        "maxTokens": max_tokens,
    }
    # Additional model inference parameters.
    additional_model_fields = {"top_k": top_k}

    system_prompts = []
    input_text = "What is the most popular song on WZPZ and WZHK? think step by step"
    messages = [{
        "role": "user",
        "content": [{"text": input_text}]
    }]
    session = boto3.session.Session(profile_name='c35',region_name='default')
    bedrock_client = session.client(service_name='bedrock-runtime')

    try:
        print(colored(f"Question: {input_text}", 'red'))
        generate_text(bedrock_client, model_id, messages,
                        system_prompts, inference_config, additional_model_fields, tool_config)

    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        print(f"A client error occured: {message}")

    else:
        print(
            f"Finished generating text with model {model_id}.")


if __name__ == "__main__":
    main()

INFO:botocore.credentials:Credentials found in config file: ~/.aws/config


Question: What is the most popular song on WZPZ and WZHK? think step by step


NameError: name 'generate_text' is not defined

## 多轮对话示例

In [7]:
import boto3
from botocore.config import Config
config = Config(read_timeout=1000) # second

# model_id = "anthropic.claude-3-haiku-20240307-v1:0"
# model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
# model_id = "anthropic.claude-3-opus-20240229-v1:0" 
model_id = "cohere.command-r-v1:0" # doesn't support tool use in streaming mode

# inference parameters to use.
temperature = 0.9
top_k = 200
max_tokens = 2000
# Base inference parameters.
inference_config = {
    "temperature": temperature,
    "maxTokens": max_tokens,
}

if model_id.startswith('anthropic.'):
    # Additional model inference parameters.
    additional_model_fields = {"top_k": top_k}
else:
    additional_model_fields = {}

class HM:
    def __init__(self, system_prompt=[]):
        self.msgs = []
        self.max_retries = 1
        self.current_num_retries = 0
        self.bedrock_client = boto3.client(service_name='bedrock-runtime', config=config)
        self.system_prompts = [{"text": sp for sp in system_prompt if sp}]

    def _put_user(self, user):
        if user:
            self.msgs.append({
                "role": "user",
                "content": [{"text": user}]
            })

    def _put_assist(self, assist):
        if assist:
            self.msgs.append({
                "role": "assistant",
                "content": [{"text": assist}]
            })

    def chat(self, user, assistant=None, stream=True):
        self._put_user(user)
        self._put_assist(assistant)

        try:
            if stream:
                stream_conversation(self.bedrock_client, model_id, self.msgs,
                            self.system_prompts, inference_config, additional_model_fields, tool_config)
            else:
                generate_text(self.bedrock_client, model_id, self.msgs,
                            self.system_prompts, inference_config, additional_model_fields, tool_config)
        except ClientError as err:
            message = err.response['Error']['Message']
            logger.error("A client error occurred: %s", message)
            print("A client error occured: " +
                format(message))
        else:
            print(
                f"\nFinished streaming messages with model {model_id}.")

    def print_msg(self):
        print(json.dumps(self.msgs, ensure_ascii=False, indent=2))

    def clear_chat(self):
        self.msgs.clear()


### Claude 3

In [8]:
hm_1 = HM()
hm_1.chat("What is the most popular song on WZPZ and WZHK?")
hm_1.chat("北京天气如何?")
hm_1.print_msg()


************ ROUND 0 START ************
Streaming messages with model cohere.command-r-v1:0


ERROR:__main__:A client error occurred: This model doesn't support tool use in streaming mode.


A client error occured: This model doesn't support tool use in streaming mode.

************ ROUND 0 START ************
Streaming messages with model cohere.command-r-v1:0


ERROR:__main__:A client error occurred: This model doesn't support tool use in streaming mode.


A client error occured: This model doesn't support tool use in streaming mode.
[
  {
    "role": "user",
    "content": [
      {
        "text": "What is the most popular song on WZPZ and WZHK?"
      }
    ]
  },
  {
    "role": "user",
    "content": [
      {
        "text": "北京天气如何?"
      }
    ]
  }
]


### Cohere Command

In [9]:
hm_2 = HM()
hm_2.chat("What is the most popular song on WZPZ and WZHK?", None, False)
hm_2.chat("北京天气如何?", None, False)
hm_2.print_msg()

INFO:__main__:Generating text with model cohere.command-r-v1:0
INFO:__main__:Requesting tool top_song. Request: tooluse_PP11vdEPSjKewrFPt2WGig
INFO:__main__:Requesting tool top_song. Request: tooluse_zKBi1vNWTPmmOL_PBl325w
INFO:__main__:Generating text with model cohere.command-r-v1:0


I will run concurrent searches for each radio station to find their most popular songs, and relay this information to the user.

Invoking tool: top_song. params: {'sign': 'WZPZ'}. id: tooluse_PP11vdEPSjKewrFPt2WGig
Tool invoke id: tooluse_PP11vdEPSjKewrFPt2WGig, result: {'json': {'song': 'Elemental Hotel', 'artist': '8 Storey Hike'}}

Invoking tool: top_song. params: {'sign': 'WZHK'}. id: tooluse_zKBi1vNWTPmmOL_PBl325w
Tool invoke id: tooluse_zKBi1vNWTPmmOL_PBl325w, result: {'json': {'song': 'Elemental Hotel', 'artist': '8 Storey Hike'}}


INFO:__main__:Generating text with model cohere.command-r-v1:0


The most popular song on both WZPZ and WZHK is Elemental Hotel by 8 Storey Hike.

Finished streaming messages with model cohere.command-r-v1:0.
I'm afraid I can't find any information about the weather in Beijing. Sorry about that!

Finished streaming messages with model cohere.command-r-v1:0.
[
  {
    "role": "user",
    "content": [
      {
        "text": "What is the most popular song on WZPZ and WZHK?"
      }
    ]
  },
  {
    "role": "assistant",
    "content": [
      {
        "text": "I will run concurrent searches for each radio station to find their most popular songs, and relay this information to the user."
      },
      {
        "toolUse": {
          "toolUseId": "tooluse_PP11vdEPSjKewrFPt2WGig",
          "name": "top_song",
          "input": {
            "sign": "WZPZ"
          }
        }
      },
      {
        "toolUse": {
          "toolUseId": "tooluse_zKBi1vNWTPmmOL_PBl325w",
          "name": "top_song",
          "input": {
            "sign": "WZHK"
 

In [18]:
# Use the Conversation API to send a text message to Anthropic Claude
# and print the response stream.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Set the model ID, e.g., Claude 3 Haiku.
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
# model_id = "meta.llama3-8b-instruct-v1:0"
model_id = "meta.llama3-2-3b-instruct-v1:0"

# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    streaming_response = client.converse_stream(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the streamed response text in real-time.
    for chunk in streaming_response["stream"]:
        if "contentBlockDelta" in chunk:
            text = chunk["contentBlockDelta"]["delta"]["text"]
            print(text, end="")

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




ERROR: Can't invoke 'meta.llama3-2-3b-instruct-v1:0'. Reason: An error occurred (ValidationException) when calling the ConverseStream operation: The provided model doesn't support on-demand throughput.


In [26]:
# Use the Conversation API to send a text message to Anthropic Claude
# and print the response stream.

import boto3
from botocore.exceptions import ClientError
from botocore.config import Config

my_config = Config(
    connect_timeout=10,  # Connection timeout in seconds
    read_timeout=120,   # Read timeout in seconds (adjust as needed)
    retries={'max_attempts': 3}  # Optional: Set max retry attempts
)
# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-west-2",config=my_config)

# Set the model ID, e.g., Claude 3 Haiku.
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
# model_id = "meta.llama3-2-3b-instruct-v1:0"
# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    streaming_response = client.converse_stream(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the streamed response text in real-time.
    for chunk in streaming_response["stream"]:
        if "contentBlockDelta" in chunk:
            text = chunk["contentBlockDelta"]["delta"]["text"]
            print(text, end="")

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




The purpose of a 'hello world' program is to serve as a simple and straightforward introduction to programming by displaying the message "Hello, World!" as the output.